# Federated Learning (IID setting)
Train a centralized model on a decentralized data.
Dataset used: CIFAR-10. In PyTorch, CIFAR 10 is available to use with the help of the torchvision module.

# To import all the relevant packages 
In this study, the dataset is randomly divided into the clients and all local models will be trained on the same machine. 

In [ ]:
###############################
##### Importing libraries #####
###############################

import os
import random
from tqdm import tqdm
import numpy as np
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data.dataset import Dataset   
torch.backends.cudnn.benchmark=True

#tqdm is a Python library that allows you to output a smart progress bar by wrapping around any iterable (create Progress bars/meters). 
#A tqdm progress bar not only shows you how much time has elapsed, but also shows the estimated time remaining for the iterable.

# Setting the Hyper-parameters
**num_clients:** Total number of clients, further this is used to divide the dataset into num_clients with every client, having the same amount of images.

**num_selected:** Number of randomly selected clients from num_clients during communication round. To be used in the training section. Generally, num_selected is around 30% of the num_clients.

**num_rounds:** Total number of communication rounds. In each communication round, num_clients are randomly selected, training on client’s devices takes place, which is followed by aggregation of the individual model weights into one global model.

**epochs:** Total number of local training rounds on each selected client’s device.

**batch_size:** Loading the data into the data loader by batches.

In [ ]:
##### Hyperparameters for federated learning #########
num_clients = 20
num_selected = 6
num_rounds = 50
epochs = 5
batch_size = 32

# Loading and dividing CIFAR-10 into clients
Images are equally divided into clients, thus representing the balanced (IID) case. CIFAR10 dataset consists of 60,000 color images of 32x32 pixels in 10 classes. There are 50,000 training images and 10,000 test images. In the training batch, there are 5,000 images from each class, which makes 50,000 in total.

In [ ]:
#############################################################
##### Creating desired data distribution among clients  #####
#############################################################

# Image augmentation 
# Define the image augmentation and normalization method for the training data to be used while loading the images. 
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


#-------------------------------------------------TRAINING-------------------------------------------------#
# Loading CIFAR10 using torchvision.datasets
# Load the training data with the given augmentation.
traindata = datasets.CIFAR10('./data', train=True, download=True,
                       transform= transform_train)

# Dividing the training data into num_clients, with each client having equal number of images
# Splits the training data into num_clients, i.e. 20 in our case.
traindata_split = torch.utils.data.random_split(traindata, [int(traindata.data.shape[0] / num_clients) for _ in range(num_clients)])

# Creating a pytorch loader for a Deep Learning model
train_loader = [torch.utils.data.DataLoader(x, batch_size=batch_size, shuffle=True) for x in traindata_split]
#to iterate through the data

#-------------------------------------------------TESTING-------------------------------------------------#
# Normalizing the test images
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Loading the test iamges and thus converting them into a test_loader
test_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10('./data', train=False, transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
        ), batch_size=batch_size, shuffle=True)


Extracting ./data/cifar-10-python.tar.gz to ./data


# Building the Neural Network Model (Model architecture)

**VGG:** It was proposed by the Visual Geometry Group of Oxford University in 2014 and obtained accurate classification performance on the ImageNet dataset.

**VGG19:** 16 convolution layers, 3 Fully Connected layers, 5 MaxPool layers (Summarizing the output of Convolution Layer), and 1 SoftMax layer (Softmax is implemented through a neural network layer just before the output layer. The Softmax layer must have the same number of nodes as the output layer).

In [ ]:
#################################
##### Neural Network model #####
#################################

#VGG is a deep CNN used to classify images.

cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

class VGG(nn.Module):
    def __init__(self, vgg_name):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Sequential(
            nn.Linear(512, 512),
            nn.ReLU(True),
            nn.Linear(512, 512),
            nn.ReLU(True),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        output = F.log_softmax(out, dim=1)
        return output

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)

# Helper functions for Federated Learning

The **client_update** function train the client model on private client data. This is the local training round that takes place at num_selected clients, i.e. 6 in our case.

In [ ]:
def client_update(client_model, optimizer, train_loader, epoch=5):
    """
    This function updates/trains client model on client data
    """
    model.train()
    for e in range(epoch):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.cuda(), target.cuda()
            optimizer.zero_grad()
            output = client_model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()
    return loss.item()

The **server_aggregate** function aggregates the model weights received from every client and updates the global model with the updated weights. Here, the mean of the weights is taken and aggregated into the global weights.

In [ ]:
def server_aggregate(global_model, client_models):
    """
    This function has aggregation method 'mean'
    """
    ### This will take simple mean of the weights of models ###
    global_dict = global_model.state_dict()
    for k in global_dict.keys():
        global_dict[k] = torch.stack([client_models[i].state_dict()[k].float() for i in range(len(client_models))], 0).mean(0)
    global_model.load_state_dict(global_dict)
    for model in client_models:
        model.load_state_dict(global_model.state_dict())

The **test** function is the standard function, which takes the global model along with the test loader as the input and returns the test loss and accuracy.

In [ ]:
def test(global_model, test_loader):
    """This function test the global model on test data and returns test loss and test accuracy """
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.cuda(), target.cuda()
            output = global_model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    acc = correct / len(test_loader.dataset)

    return test_loss, acc

# Training the Model
One global model, along with the individual client_models is initialized with VGG19 on a GPU. Here, SGD is used as an optimizer for all the client models.

In [ ]:
############################################
#### Initializing models and optimizer  ####
############################################

#### global model ##########
global_model =  VGG('VGG19').cuda()

############## client models ##############
client_models = [ VGG('VGG19').cuda() for _ in range(num_selected)]
for model in client_models:
    model.load_state_dict(global_model.state_dict()) ### initial synchronizing with global model 

############### optimizers ################
opt = [optim.SGD(model.parameters(), lr=0.1) for model in client_models]

In [ ]:
###### List containing info about learning #########
#create a list for keeping a track of the loss and accuracy for the train and test dataset
losses_train = []
losses_test = []
acc_train = []
acc_test = []

# Runnining FL
'''Training of individual clients starts, i.e. the communication round. 
Initially, num_selected clients are selected from num_clients, i.e. 6 clients are randomly selected from 20 available clients. 
Training takes place for every selected client using the client_update function. '''

for r in range(num_rounds):
    # select random clients
    client_idx = np.random.permutation(num_clients)[:num_selected]
    # client update
    loss = 0
    for i in tqdm(range(num_selected)):
        loss += client_update(client_models[i], opt[i], train_loader[client_idx[i]], epoch=epochs)
    
    losses_train.append(loss)
    # server aggregate
    '''Now, the aggregation of the weights takes place using the server_aggregate function. 
    This updates the global model, which is the final model that is used for prediction. After updating the global 
    model, this global model is used to test the training with the help of the test function defined above.'''

    server_aggregate(global_model, client_models)
    
    test_loss, acc = test(global_model, test_loader)
    losses_test.append(test_loss)
    acc_test.append(acc)
    print('%d-th round' % r)
    print('average train loss %0.3g | test loss %0.3g | test acc: %0.3f' % (loss / num_selected, test_loss, acc))

#This process continues for num_rounds, i.e. 150 communication rounds in our case.

  0%|          | 0/6 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
  0%|          | 0/6 [00:00<?, ?it/s]

0-th round
average train loss 1.83 | test loss 2.07 | test acc: 0.263


  0%|          | 0/6 [00:00<?, ?it/s]

1-th round
average train loss 1.74 | test loss 1.73 | test acc: 0.328


  0%|          | 0/6 [00:00<?, ?it/s]

...

47-th round
average train loss 0.197 | test loss 0.442 | test acc: 0.875


  0%|          | 0/6 [00:00<?, ?it/s]

48-th round
average train loss 0.4 | test loss 0.48 | test acc: 0.861


100%|██████████| 6/6 [01:03<00:00, 10.54s/it]


49-th round
average train loss 0.991 | test loss 0.501 | test acc: 0.856
